In [247]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy.exceptions import AxisError
import pandas as pd
from scipy import stats

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Integer, String, insert, select

engine = create_engine("sqlite+pysqlite:///GPSdata1.db", echo=False)

metadata_obj = MetaData()

players_table = Table("players", metadata_obj, autoload_with=engine)
sessions_data_table = Table("sessions_data", metadata_obj, autoload_with=engine)

In [248]:
measure = "HSR/min"
measures = sessions_data_table.columns
print(measures.keys())

['player_id', 'match_id', 'Duration', 'Distance', 'HSR', 'SD', 'Accels + Decels', 'PlayerLoad', 'Top Speed', 'Dis/min', 'HSR/min', 'SD/min', 'Accels + Decels /min']


In [249]:
measure = "HSR/min"
measures = sessions_data_table.columns
HSR_columns = measures[measure]

with engine.connect() as conn:
    stmt = (
        select(players_table.c.team, players_table.c.position, measures)
        .join(sessions_data_table, players_table.c.id == sessions_data_table.c.player_id)
        .order_by(players_table.c.team)
    )

    data = conn.execute(stmt).mappings().all()

df = pd.DataFrame(data)


In [250]:
positions = 4

In [251]:
pd.DataFrame({"mean": [df["HSR/min"].mean()]})

,mean
0,5.867944


In [252]:
# dataframe = df

# table_mean = pd.pivot_table(dataframe, values=measure,
#                         columns=['team', 'position'], aggfunc="mean").round(decimals=2)

# table_mean_team = pd.pivot_table(dataframe, values=measure,
#                     columns=['team'], aggfunc="mean").round(decimals=2)

# table_mean_all = pd.DataFrame({"Overall": [df["HSR/min"].mean()]})

# table_mean.insert(positions + ((positions + 1) * i), "TEAM", table_mean_team[team], allow_duplicates=True)
# table_mean.insert(positions * 4, "Overall", df["HSR/min"].mean(), allow_duplicates=True)
# table_mean

In [257]:
# Produces df with a measure's team and position means

def generate_mean_sd_table(measure: str, dataframe: pd.DataFrame):

    table_mean = pd.pivot_table(dataframe, values=measure,
                        columns=['team', 'position'], aggfunc="mean").round(decimals=2)

    table_mean_team = pd.pivot_table(dataframe, values=measure,
                        columns=['team'], aggfunc="mean").round(decimals=2)

    # table_mean_all = pd.DataFrame({"Overall": [df["HSR/min"].mean()]})

    table_std = pd.pivot_table(dataframe, values=measure,
                        columns=['team', 'position'], aggfunc="std").round(decimals=2)

    table_std_team = pd.pivot_table(dataframe, values=measure,
                        columns=['team'], aggfunc="std").round(decimals=2)
    
    # table_std_all = pd.DataFrame({"Overall": [df["HSR/min"].std()]})

    for i, team in enumerate(table_mean_team):
        table_mean.insert(positions + ((positions + 1) * i), "TEAM", table_mean_team[team], allow_duplicates=True)
        table_std.insert(positions + ((positions + 1) * i), "TEAM", table_std_team[team], allow_duplicates=True)
        

    table_mean.insert(positions * 5, "Overall", df[measure].mean(), allow_duplicates=True)
    table_std.insert(positions * 5, "Overall", df[measure].std(), allow_duplicates=True)


    table_measure_std_mean = pd.concat([table_mean, table_std])
    table_measure_std_mean.insert(0,"", ["Mean", "SD"], allow_duplicates=True)
    
    return table_measure_std_mean

In [258]:
measures = ['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad']

sub_tables = []

for measure in measures:
    sub_tables.append(generate_mean_sd_table(measure, df))

        
table = pd.concat(sub_tables)
table



team                           1st                            TEAM     U17  \
position                        CB      CM      FB     FWD              CB   
HSR/min               Mean    3.54    7.76    6.79    6.68    6.19    3.18   
HSR/min                 SD    0.71    1.98    1.65    2.03    2.29    0.98   
Dis/min               Mean   96.28  122.49  110.38  104.74  108.22  100.13   
Dis/min                 SD    2.82    7.00    6.57    8.09   11.31    4.68   
SD/min                Mean    0.90    2.08    2.24    2.75    2.01    0.31   
SD/min                  SD    0.36    1.32    0.78    1.23    1.20    0.27   
Accels + Decels /min  Mean    1.42    2.38    2.21    1.95    1.99    1.62   
Accels + Decels /min    SD    0.16    0.36    0.27    0.57    0.52    0.30   
PlayerLoad            Mean  384.01  519.87  484.05  433.74  454.40  392.72   
PlayerLoad              SD   20.67   36.15   20.14   35.06   58.11   25.32   

team                                          ...     U18                  \
position                  CM      FB     FWD  ...      CM      FB     FWD   
HSR/min                 5.58    7.21    7.61  ...    4.78    7.10    7.16   
HSR/min                 2.45    0.92    1.86  ...    1.89    1.60    1.54   
Dis/min               114.67  114.84  107.99  ...  116.22  115.06  115.21   
Dis/min                 8.29    3.98    8.00  ...    6.04    5.87   12.92   
SD/min                  0.46    1.21    2.32  ...    0.65    1.98    2.20   
SD/min                  0.31    0.23    1.53  ...    0.47    0.63    0.79   
Accels + Decels /min    2.19    2.69    1.94  ...    1.77    2.13    2.25   
Accels + Decels /min    0.47    0.32    0.42  ...    0.49    0.36    0.43   
PlayerLoad            465.24  481.00  424.96  ...  480.83  522.00  462.69   
PlayerLoad             45.88   15.61   50.58  ...   29.19   24.59   33.85   

team                    TEAM     U23                            TEAM  \
position                          CB      CM      FB     FWD           
HSR/min                 5.40    4.06    5.43    7.29    6.94    5.88   
HSR/min                 1.81    0.76    1.59    1.64    1.38    1.83   
Dis/min               111.35  102.83  107.82  110.07  101.97  105.82   
Dis/min                 8.99    5.68    7.58    7.07    7.25    7.73   
SD/min                  1.13    0.77    0.97    2.41    2.30    1.52   
SD/min                  0.80    0.43    0.70    1.25    1.01    1.12   
Accels + Decels /min    1.93    1.51    2.04    2.13    2.12    1.98   
Accels + Decels /min    0.42    0.25    0.37    0.39    0.51    0.45   
PlayerLoad            458.49  422.03  456.45  458.48  387.07  432.95   
PlayerLoad             41.34   29.50   39.61   43.55   33.39   47.40   

team                     Overall  
position                          
HSR/min                 5.867944  
HSR/min                 2.061575  
Dis/min               108.167354  
Dis/min                 9.491179  
SD/min                  1.519275  
SD/min                  1.149754  
Accels + Decels /min    1.980070  
Accels + Decels /min    0.473452  
PlayerLoad            445.366730  
PlayerLoad             51.010119  

[10 rows x 22 columns]

In [255]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [259]:
wb = Workbook()
ws = wb.active
ws.title = "Data table"

rows = dataframe_to_rows(table)

for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)

wb.save("test.xlsx")


In [263]:
df.describe()["HSR/min"]

count    563.000000
mean       5.867944
std        2.061575
min        0.794844
25%        4.302262
50%        5.661481
75%        7.337832
max       12.896371
Name: HSR/min, dtype: float64